In [2]:
import plotly
# import plotly.plotly as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import mysql.connector
import config

In [3]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

In [4]:
stmt = "SELECT * from nba.shot_chart"
cursor.execute(stmt)
scores = cursor.fetchall()

field_names = [i[0] for i in cursor.description]
shots = pd.DataFrame(scores)
shots.columns = field_names

In [5]:
shots.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [6]:
shots[shots['PLAYER_NAME'] == 'Ben Simmons']

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
403975,Shot Chart Detail,0021700006,33,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,9,44,...,Center(C),Less Than 8 ft.,3,-18,26,1,0,20171018,WAS,PHI
403976,Shot Chart Detail,0021700006,47,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,8,28,...,Center(C),Less Than 8 ft.,1,12,11,1,1,20171018,WAS,PHI
403977,Shot Chart Detail,0021700006,77,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,5,54,...,Right Side(R),8-16 ft.,9,73,55,1,1,20171018,WAS,PHI
403978,Shot Chart Detail,0021700006,182,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,11,43,...,Center(C),Less Than 8 ft.,0,0,-6,1,1,20171018,WAS,PHI
403979,Shot Chart Detail,0021700006,187,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,10,42,...,Center(C),Less Than 8 ft.,0,8,1,1,1,20171018,WAS,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406069,Shot Chart Detail,0021900230,73,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,5,43,...,Center(C),Less Than 8 ft.,1,3,13,1,1,20191123,PHI,MIA
406070,Shot Chart Detail,0021900230,301,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,2,19,...,Center(C),Less Than 8 ft.,3,-4,32,1,0,20191123,PHI,MIA
406071,Shot Chart Detail,0021900230,394,1627732,Ben Simmons,1610612755,Philadelphia 76ers,3,8,10,...,Center(C),Less Than 8 ft.,6,-30,59,1,0,20191123,PHI,MIA
406072,Shot Chart Detail,0021900230,442,1627732,Ben Simmons,1610612755,Philadelphia 76ers,3,3,58,...,Center(C),Less Than 8 ft.,6,-5,62,1,0,20191123,PHI,MIA


In [7]:
fig = go.Figure(data=go.Scatter(x=shots[shots['PLAYER_NAME']=='Ben Simmons']['LOC_X'],
                                y=shots[shots['PLAYER_NAME']=='Ben Simmons']['LOC_Y'],
                                mode = 'markers'
                     )
             )
fig.update_xaxes(range=[0, 7])
fig.update_yaxes(range=[0, 2.5])
fig.show()

In [8]:
string = '2017'
string = string[0] + string[2:]
string

'217'

In [9]:
def shot_chart(name, year=''):    
    if year:
        year_clean = '00' + year[0] + year[2:]
        player = shots[(shots.PLAYER_NAME == name) & (shots.GAME_ID.str.contains(year_clean))]
    else:
        player= shots[shots.PLAYER_NAME == name]

    missed_shot_trace = go.Scattergl(
            x = player[player.SHOT_MADE_FLAG == 0]['LOC_X'],
            y = player[player.SHOT_MADE_FLAG == 0]['LOC_Y'],
            mode = 'markers',
            name = 'Make',
            marker= dict(color='blue', symbol='x', size=8, line={'width':1}, opacity=0.7),
            text = [str(sd) for sd in player[player.SHOT_MADE_FLAG == 0]['ACTION_TYPE']],
            hoverinfo = 'text'
        )
    made_shot_trace = go.Scattergl(
        x = player[player.SHOT_MADE_FLAG == 1]['LOC_X'],
        y = player[player.SHOT_MADE_FLAG == 1]['LOC_Y'],
        mode = 'markers',
        name='Make',
        marker= dict(color='red', symbol='circle', size=8, line={'width':1}, opacity=0.7),
        text = [str(sd) for sd in player[player.SHOT_MADE_FLAG == 1]['ACTION_TYPE']],
        hoverinfo = 'text'
    )

    
   
    data = [missed_shot_trace, made_shot_trace]
    layout = go.Layout(
        title= name + ' Shot Chart ' + year,
        showlegend =True,
        xaxis={'showgrid':False, 'range':[-250,250]},
        yaxis={'showgrid':False, 'range':[-47.5,500]},
        height = 600,
        width = 650)
    #     shapes=court_shapes)

    fig = go.Figure(data=data, layout=layout)
    fig.show()

In [10]:
shot_chart('Ben Simmons', '2017')

In [11]:
def shot_breakdown(name, year = ''):    
    
    if year:
        year_clean = '00' + year[0] + year[2:]
        player = shots[(shots.PLAYER_NAME == name) & (shots.GAME_ID.str.contains(year_clean))]
    else:
        player= shots[shots.PLAYER_NAME == name]
        
    player_dummy = pd.get_dummies(player, columns=['ACTION_TYPE', 'SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])
    
    shot_breakdown = player_dummy.drop(labels=['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM', '2PT Field Goal', '3PT Field Goal'], axis=1).mean()
    
    top_10_types = shot_breakdown.sort_values(ascending=False)[:10]
    
    data = go.Bar(x=top_10_types.index, 
                  y=top_10_types.values)
    layout = go.Layout(
                        title = name + ' Percentage of Shots Taken ' + year)
    
    fig = go.Figure(data=data, layout = layout)
    fig.show()
    

In [12]:
shot_breakdown('Ben Simmons', '2017')

In [13]:
shot_chart('Chris Paul', '2017')
shot_breakdown('Chris Paul', '2017')

In [77]:
shot_chart('JJ Redick', '2017')
shot_breakdown('JJ Redick', '2017')

In [79]:
shot_chart('LaMarcus Aldridge', '2017')
shot_breakdown('LaMarcus Aldridge', '2017')

In [82]:
shot_chart('James Harden', '2017')
shot_breakdown('James Harden', '2017')

In [54]:
sim17 = shots[(shots['PLAYER_NAME'] == 'Ben Simmons') & (shots.GAME_ID.str.contains('00217'))]
sim_dummy = pd.get_dummies(sim17, columns=['ACTION_TYPE','SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])
sim_dummy.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'Alley Oop Dunk Shot', 'Alley Oop Layup shot', 'Cutting Dunk Shot',
       'Cutting Finger Roll Layup Shot', 'Cutting Layup Shot',
       'Driving Dunk Shot', 'Driving Finger Roll Layup Shot',
       'Driving Floating Bank Jump Shot', 'Driving Floating Jump Shot',
       'Driving Hook Shot', 'Driving Layup Shot', 'Driving Reverse Dunk Shot',
       'Driving Reverse Layup Shot', 'Dunk Shot', 'Fadeaway Jump Shot',
       'Finger Roll Layup Shot', 'Floating Jump shot', 'Hook Shot',
       'Jump Bank Shot', 'Jump Shot', 'Layup Shot', 'Pullup Jump shot',
       'Putback Dunk Shot', 'Putback Layup Shot', 'Reverse Dunk Shot',
       'Rever

In [57]:
shot_breakdown = sim_dummy.drop(labels=['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM', '2PT Field Goal', '3PT Field Goal'], axis=1).mean()

In [59]:
shot_breakdown.sort_values(ascending=False)[:10]

Driving Layup Shot                0.136273
Jump Shot                         0.089178
Layup Shot                        0.083166
Pullup Jump shot                  0.068136
Driving Dunk Shot                 0.045090
Turnaround Hook Shot              0.045090
Dunk Shot                         0.038076
Driving Finger Roll Layup Shot    0.036072
Driving Hook Shot                 0.036072
Driving Floating Jump Shot        0.034068
dtype: float64